In [1]:
import torch


ModuleNotFoundError: No module named 'torch'

In [ ]:
import numpy as np

# Path to GloVe file
glove_path = "../assets/glove.6B.300d.txt"  # Adjust the file name for the desired dimensions

def load_glove_embeddings(glove_file):
    """
    Load GloVe into Python GloVe files are in plain text format. Convert them into a dictionary 
    """
    embeddings_index = {}
    with open(glove_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector
    return embeddings_index

# Load the embeddings
glove_embeddings = load_glove_embeddings(glove_path)
print(f"Loaded {len(glove_embeddings)} word vectors.")


Loaded 400000 word vectors.


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)  # X_train contains your training data

vocab_size = len(tokenizer.word_index) + 1
max_len = 100  # Maximum sentence length
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len)
